## Imports

In [1]:
import numpy as np
import sys, os
from numba import njit

### Methods

In [2]:
@njit
def energy(gs,stiff,gs_sum,stiff_sum,X):
    dX = X - gs
    E = 0.5 * dX.T @ stiff @ dX
    E +=  0.5 * (np.sum(X)-gs_sum)**2 * stiff_sum
    return E

def sping_system(gs, stiff, gs_sum, stiff_sum, samples, steps_per_sample,initial = None):
    dim = len(gs)
    
    if initial is None:
        X = np.copy(gs)
    else:
        X = np.copy(initial)
    E_cur = energy(gs,stiff,gs_sum,stiff_sum,X)
        
    X_sample = np.empty((samples,len(gs)))
    cov = np.linalg.inv(stiff)
    
    scov = np.zeros((dim,dim))
    cnt = 0
    
    for s in range(samples):
        print(s)
        dX = np.random.multivariate_normal(np.zeros(cov.shape[0]), cov*0.5, steps_per_sample)
        
        X,E_cur,scov,cnt = cal_nsteps(dX,X,gs,stiff,gs_sum,stiff_sum,E_cur,scov,cnt)
        
        # for dx in dX:
        #     Y = X + dx
        #     E_new = energy(gs,stiff,gs_sum,stiff_sum,Y)
        #     dE = E_new-E_cur
        #     # metropolis step
        #     if np.random.uniform() < np.exp(-dE):
        #         E_cur = E_new
        #         X = Y 
            
        #     dX = X-gs
        #     scov += np.outer(dX,dX)
        #     cnt  += 1
            
        X_sample[s] = X
    
    scov = scov / cnt
    return X_sample,scov

@njit
def cal_nsteps(dX,X,gs,stiff,gs_sum,stiff_sum,E_cur,scov,cnt):
    for dx in dX:
        Y = X + dx
        E_new = energy(gs,stiff,gs_sum,stiff_sum,Y)
        dE = E_new-E_cur
        # metropolis step
        if np.random.uniform() < np.exp(-dE):
            E_cur = E_new
            X = Y 
        
        dX = X-gs
        scov += np.outer(dX,dX)
        cnt  += 1
    return X,E_cur,scov,cnt

@njit
def covmat(vecs):
    dim = vecs.shape[-1]
    cov = np.zeros((dim,dim))
    for i in range(len(vecs)):
        cov += np.outer(vecs[i],vecs[i])
    cov /= len(vecs)
    return cov

### Define System and sample

In [16]:
k0 = 5
k1 = 0.5
k2 = 0.1
dim = 4

x0 = 1

gs_sum = dim*x0
stiff_sum = 2

gs = np.ones(dim)*x0
M = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        if i == j:
            M[i,j] = k0
        if np.abs(i-j) == 1:
            M[i,j] = k1
        if np.abs(i-j) == 2:
            M[i,j] = k2
            
print(M)
print(gs)

samples = 100
steps_per_sample = 1000000

Xs,scov = sping_system(gs, M, gs_sum, stiff_sum, 1, 1000)     
Xs,scov = sping_system(gs, M, gs_sum, stiff_sum, samples, steps_per_sample,initial = Xs[0])  

[[5.  0.5 0.1 0. ]
 [0.5 5.  0.5 0.1]
 [0.1 0.5 5.  0.5]
 [0.  0.1 0.5 5. ]]
[1. 1. 1. 1.]
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


### Evaluate

In [19]:
np.set_printoptions(linewidth=250, precision=2, suppress=True,edgeitems=12)

Mtot = np.linalg.inv(scov)
print('Sampled Mtot:')    
print(Mtot)

T = np.eye(4)
T[3,0] = 1
T[3,1] = 1
T[3,2] = 1
Ti = np.linalg.inv(T)

print('T:')
print(T)
print('T_inv:')
print(Ti)

M_alt    = Ti.T @ M @ Ti
Mtot_alt = Ti.T @ Mtot @ Ti

print('M:')
print(M)
print('M_alt:')
print(M_alt)
print('Mtot_alt:')
print(Mtot_alt)

M_diff = Mtot_alt - M_alt
print('Mdiff:')
print(M_diff)

Sampled Mtot:
[[7.01 2.5  2.1  2.  ]
 [2.5  7.   2.5  2.1 ]
 [2.1  2.5  7.   2.5 ]
 [2.   2.1  2.5  7.  ]]
T:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 1. 1. 1.]]
T_inv:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [-1. -1. -1.  1.]]
M:
[[5.  0.5 0.1 0. ]
 [0.5 5.  0.5 0.1]
 [0.1 0.5 5.  0.5]
 [0.  0.1 0.5 5. ]]
M_alt:
[[10.   5.4  4.6 -5. ]
 [ 5.4  9.8  4.9 -4.9]
 [ 4.6  4.9  9.  -4.5]
 [-5.  -4.9 -4.5  5. ]]
Mtot_alt:
[[10.   5.4  4.6 -5. ]
 [ 5.4  9.8  4.9 -4.9]
 [ 4.6  4.9  9.  -4.5]
 [-5.  -4.9 -4.5  7. ]]
Mdiff:
[[ 0. -0.  0. -0.]
 [-0. -0. -0. -0.]
 [ 0. -0. -0. -0.]
 [-0. -0. -0.  2.]]
